In [154]:
from curses import flash
import os
from pickle import TRUE
from re import L
import requests
import validators
import gzip
import shutil
from urllib.parse import urlparse

import sys
import math
import xarray as xa
import numpy as np
from rio_cogeo import cog_validate
import rioxarray

# Mapping
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [155]:
#main variables
base_path = '/home/asubedi/Desktop/hs3_temp_repo'
file_name = 'AE20141015.Gonzalo.loc.nc'
path = f'{base_path}/{file_name}'
engine = 'netcdf4'
var_name = 'Subset_Of_Stations'
split_range = 10000
current_setting = "Full"

In [156]:
#open file here
file1 = xa.open_dataset(path, engine=engine, decode_coords='all', decode_times=False)
print(file1)

<xarray.Dataset>
Dimensions:             (time: 17979)
Dimensions without coordinates: time
Data variables:
    Time                (time) float64 ...
    Latitude            (time) float32 ...
    Longitude           (time) float32 ...
    Residual_Fit_Error  (time) float32 ...
    Number_Of_Stations  (time) int32 ...
    Energy              (time) float32 ...
    Energy_Uncertainty  (time) float32 ...
    Subset_Of_Stations  (time) int32 ...
Attributes:
    Conventions:                       CF-1.6
    Title:                             Hurricane and Severe Storm Sentinel (H...
    Description:                       WWLLN stroke data subset for Tropical ...
    Institution:                       World Wide Lightning Location Network,...
    Source:                            Lightning location data from the World...
    History:                           Observations from the World Wide Light...
    References:                        Please acknowledge the WWLLN network a...
    Comm

In [157]:
#lat, lon, data
latitude = file1.Latitude.data
longitude = file1.Longitude.data
data = file1[var_name].data

In [158]:
print(latitude, longitude, data)

[21.7249 21.7501 20.9063 ... 23.8441 26.1161 26.058 ] [-66.3656 -66.3943 -62.2227 ... -45.0211 -44.8177 -44.8133] [ 8  6 11 ... 10  6  8]


In [159]:
#split data/geolocation here -- if needed

split_lat = (np.copy(latitude[0:split_range]))
split_lon = (np.copy(longitude[0:split_range]))
split_data = (np.copy(values[0:split_range]))

In [160]:
split_dict = []
for i in range(len(split_data)):
    split_dict.append({
        "latitude":split_lat[i], 
        "longitude":split_lon[i],
        "data":split_data[i]
    })
print("Split_Dict Complete!!", len(split_dict))    

full_dict = []
for i in range(len(data)):
    full_dict.append({
        "latitude":latitude[i],
        "longitude":longitude[i],
        "data":data[i]
    })

print("Full_Dict Complete!!", len(full_dict))

Split_Dict Complete!! 10000
Full_Dict Complete!! 17979


In [161]:
#Sort geolocation here
full_sorted_lat = np.sort(np.copy(latitude), axis=0)
full_sorted_lon = np.sort(np.copy(longitude), axis=0)

split_sorted_lat = np.sort(np.copy(split_lat), axis=0)
split_sorted_lon = np.sort(np.copy(split_lon), axis = 0)

In [162]:
#init grid here
full_grid = np.zeros((len(latitude), len(latitude)))
split_grid = np.zeros((split_range, split_range))

In [163]:
def find_full_lat_lon(lat, lon):
    lat_index = None
    lon_index = None
    for i in range(len(full_sorted_lat)):
        if(full_sorted_lat[i] == lat):
            lat_index = i
            break;
    for i in range(len(full_sorted_lon)):
        if(full_sorted_lon[i] == lon):
            lon_index = i
            break;
    #print(lat_index, lon_index)
    return [lat_index, lon_index]
            
def find_split_lat_lon(lat, lon):
    lat_index = None
    lon_index = None
    for i in range(len(split_sorted_lat)):
        if(split_sorted_lat[i] == lat):
            lat_index = i
            break;
    for i in range(len(split_sorted_lon)):
        if(split_sorted_lon[i] == lon):
            lon_index = i
            break;
    return [lat_index, lon_index]

In [164]:
for di in full_dict:
    index = find_full_lat_lon(di["latitude"], di["longitude"])
    full_grid[index[0]][index[1]] = di["data"]

for di in split_dict:
    index = find_split_lat_lon(di["latitude"], di["longitude"])
    split_grid[index[0]][index[1]] = di["data"]

In [165]:
if(current_setting == 'Split'):
    new_xarray = xa.DataArray(
        data = split_grid,
        dims=("latitude", "longitude"),
        coords={
            "latitude":split_sorted_lat,
            "longitude":split_sorted_lon
        },
        attrs=dict(
            description="",
            units="",
        ),
    )
elif (current_setting == 'Full'):
    new_xarray = xa.DataArray(
        data = full_grid,
        dims=("latitude", "longitude"),
        coords={
            "latitude":full_sorted_lat,
            "longitude":full_sorted_lon
        },
        attrs=dict(
            description="",
            units="",
        ),
    )
else:
    print("Please select proper current_setting")

In [169]:
new_xarray = new_xarray.transpose('latitude', 'longitude')
new_xarray.rio.set_spatial_dims(x_dim='longitude', y_dim='latitude', inplace=True)
new_xarray.rio.crs
new_xarray.rio.set_crs('epsg:4326', inplace=TRUE)
#new_xarray.plot()
# cog_path = f'{base_path}/test-cog/alan.tif'
# new_xarray.rio.to_raster(rf'{cog_path}', driver='COG')


In [171]:
ppath = base_path + "/alan_hs3.nc"
file2 = xa.open_dataset(ppath, engine=engine, decode_coords='all', decode_times=False)

In [172]:
print(file2)

<xarray.Dataset>
Dimensions:                        (latitude: 17979, longitude: 17979)
Coordinates:
  * latitude                       (latitude) float32 16.4 16.4 ... 48.22 50.58
  * longitude                      (longitude) float32 -68.7 -68.68 ... -32.09
Data variables:
    __xarray_dataarray_variable__  (latitude, longitude) float64 ...


In [176]:
print(file2.__xarray_dataarray_variable__.data)

KeyboardInterrupt: 